In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/Othercomputers/Laptop/FINANCIAL_INFORMATION

In [3]:
# !pip install -q -r requirements.txt
# !pip install --upgrade -q transformers

In [4]:
# !huggingface-cli login

In [5]:
# Model
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig, LlamaForCausalLM
from langchain_huggingface import HuggingFacePipeline
import torch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
# Vector stores
import fitz  # PyMuPDF
import pdfplumber
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, KonlpyTextSplitter, MarkdownHeaderTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
#from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain_community.document_loaders import PyMuPDFLoader, PDFPlumberLoader ,UnstructuredPDFLoader
from langchain_community.retrievers import BM25Retriever, KNNRetriever
from langchain.retrievers import EnsembleRetriever
#from langchain_teddynote.retrievers import KiwiBM25Retriever, OktBM25Retriever
from langchain_teddynote.retrievers import OktBM25Retriever
from langchain.docstore.document import Document
#from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import unicodedata
import pymupdf4llm
#import time
#import re
#from konlpy.tag import Okt
#from pdf2image import convert_from_path
import pytesseract
from konlpy.tag import Kkma
# etc
#import os
import pandas as pd
from tqdm import tqdm
import unicodedata
import logging
#from PyPDF2 import PdfReader
#import json
from seed_module import seed_everything

seed_everything(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'  # GPU 사용 가능 여부 및 MPS 지원 여부 확인
print(device)

c:\ProgramData\anaconda3\envs\llm_titan\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [6]:
# intfloat/multilingual-e5-small
# jhgan/ko-sroberta-multitask

def get_embedding():

    embeddings = HuggingFaceEmbeddings(
        model_name='jhgan/ko-sroberta-multitask',
        model_kwargs={'device': device},

        encode_kwargs={'normalize_embeddings': True})
    return embeddings
def normalize_string(s):
    try:
        normalized = unicodedata.normalize('NFC', s)
        return normalized.encode('utf-8', errors='replace').decode('utf-8')
    except Exception:
        return s
def clean_text(text):
    text = text.replace("�", " ").replace("", " ")  # 잘못된 인코딩 문자 제거
    return text

def format_docs(docs):
    context = ""

    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header{header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context


In [7]:
def process_pdf(pdf_path):
    md_text = pymupdf4llm.to_markdown(pdf_path)

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
        ("####", "Header 4"),
        ("#####", "Header 5"),
        ("######", "Header 6")
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=True)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300, chunk_overlap=60
    )

    splits = text_splitter.split_documents(md_chunks)
    

    for i in splits:
        
        metadata = {'Source_path': pdf_path}
        i.metadata = {**i.metadata, **metadata}
    return splits


def make_db(df):
    documents = []

    pdf_files = df['Source_path'].unique()
    for pdf_file in tqdm(pdf_files):
        # 문서 로드
        documents.extend(process_pdf(pdf_file))

    print(f"Total number of documents: {len(documents)}")

    faiss = FAISS.from_documents(documents, embedding=get_embedding())
    return faiss

In [8]:
def fewshot_db(df):
    df = df.drop('SAMPLE_ID', axis=1)
    df = df.drop('Source_path', axis=1)
    df = df.to_dict(orient='records')
    print("Loaded Fewshot Set:", len(df))
    to_vectorize = ["\n\n".join(normalize_string(value) for value in example.values()) for example in df]
    faiss = FAISS.from_texts(to_vectorize, embedding=get_embedding())
    # bm = BM25Retriever.from_texts(to_vectorize)
    # knn = KNNRetriever.from_texts(to_vectorize, embeddings=get_embedding())
    return faiss

In [9]:
train_df = pd.read_csv('train.csv', encoding='utf-8')
test_df = pd.read_csv('test.csv', encoding='utf-8')

In [10]:
def format_docs(docs):
    context = ""
    for doc in docs:
        # Header 정보를 순서대로 추가
        for header_level in range(1, 6):
            header_key = f'Header {header_level}'
            if header_key in doc.metadata:
                context += f"{header_key}: {doc.metadata[header_key]}\n"
        # 문서 내용 추가
        context += doc.page_content
        context += '\n---\n'
    return context

### Model

In [11]:
def setup_llm_pipeline(model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"):
    # 토크나이저 로드 및 설정
        # 양자화 설정 적용
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config,low_cpu_mem_usage=True)

    # # 일부 중요한 레이어는 FP16으로 유지
    # for name, module in model.named_modules():
    #     if "attention" in name or "ffn" in name:  # 중요한 레이어 식별 (예: attention 및 ffn)
    #         module.to(torch.float16)  # 이 부분은 16비트로 유지

    tokenizer = AutoTokenizer.from_pretrained(model_id, clean_up_tokenization_spaces=True)
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        return_full_text=False,
        max_new_tokens=1024,
        eos_token_id = terminators,
        pad_token_id = tokenizer.eos_token_id
    )

    llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return llm
# ghost-x/ghost-8b-beta-1608
# OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
llm = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 4/4 [00:21<00:00,  5.28s/it]


In [12]:
# from transformers import QuantoConfig

# def setup_llm_pipeline(model_id="hugging-quants/Meta-Llama-3.1-8B-Instruct-GPTQ-INT4"):

#     # 모델 로드 및 양자화 적용
#     model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16,
#     low_cpu_mem_usage=True,
#     device_map="auto",
#     )

#     tokenizer = AutoTokenizer.from_pretrained(model_id)
#     terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
#     ]

#     text_generation_pipeline = pipeline(
#         model=model,
#         tokenizer=tokenizer,
#         task="text-generation",
#         return_full_text=False,
#         max_new_tokens=1024,
#         eos_token_id = terminators,
#         pad_token_id = tokenizer.eos_token_id
#     )

#     llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

#     return llm
# # ghost-x/ghost-8b-beta-1608
# # OpenBuddy/openbuddy-llama3.1-8b-v22.3-131k
# llm = setup_llm_pipeline()

### 점수

In [13]:
from collections import Counter
def calculate_f1_score(true_sentence, predicted_sentence, sum_mode=True):

    #공백 제거
    true_sentence = ''.join(true_sentence.split())
    predicted_sentence = ''.join(predicted_sentence.split())

    true_counter = Counter(true_sentence)
    predicted_counter = Counter(predicted_sentence)

    #문자가 등장한 개수도 고려
    if sum_mode:
        true_positive = sum((true_counter & predicted_counter).values())
        predicted_positive = sum(predicted_counter.values())
        actual_positive = sum(true_counter.values())

    #문자 자체가 있는 것에 focus를 맞춤
    else:
        true_positive = len((true_counter & predicted_counter).values())
        predicted_positive = len(predicted_counter.values())
        actual_positive = len(true_counter.values())

    #f1 score 계산
    precision = true_positive / predicted_positive if predicted_positive > 0 else 0
    recall = true_positive / actual_positive if actual_positive > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

def calculate_average_f1_score(true_sentences, predicted_sentences):

    total_precision = 0
    total_recall = 0
    total_f1_score = 0

    for true_sentence, predicted_sentence in zip(true_sentences, predicted_sentences):
        precision, recall, f1_score = calculate_f1_score(true_sentence, predicted_sentence)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score

    avg_precision = total_precision / len(true_sentences)
    avg_recall = total_recall / len(true_sentences)
    avg_f1_score = total_f1_score / len(true_sentences)

    return {
        'average_precision': avg_precision,
        'average_recall': avg_recall,
        'average_f1_score': avg_f1_score
    }

### RUN

In [14]:

def extract_answer(response):
    # AI: 로 시작하는 줄을 찾아 그 이후의 텍스트만 추출
    lines = response.split('\n')
    for line in lines:
        line = line.replace('**', '')
        if line.startswith('Answer:'):
            return line.replace('Answer:', '').strip()
        if line.startswith('assistant:'):
            return line.replace('assistant:', '').strip()
    return response.strip()  # AI: 를 찾지 못한 경우 전체 응답을 정리해서 반환

def rerun(question,context,answer,llm,num_repeat):
    full_template = "<|begin_of_text|>"
    full_template += """<|start_header_id|>system<|end_header_id|>당신은 이전 답변을 검증하는 챗봇입니다. 질문과 문맥, 이전 답변을 참고해서 지시사항을 따르세요. 지시사항을 따를 때 서론 없이 출력하세요.<|eot_id|>"""
    full_template += f"""<|start_header_id|>user<|end_header_id|>Question: {question} \n\nContexts: {context} \n\nPrevious Answer: {answer} \n\n"""
    full_template += """{input}<|eot_id|>"""
    full_template += """<|start_header_id|>assistant<|end_header_id|>"""

    prompt = PromptTemplate(template=full_template)
    chain = (
    {
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )
    torch.cuda.empty_cache()
    return chain.invoke("핵심 단어들을 바탕으로, 한 문장으로 요약하세요. 만약 한 문장이라면 그대로 출력하세요.")

def run(faiss,dataset,llm,k=3,verbose=False):
    results = []
    source_path = dataset.iloc[0]['Source_path']
    docs = faiss.similarity_search(
    query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
    filter={"Source_path": source_path},
    k = 99,
    fetch_k = 20000
    )
    buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
    faiss_retriever_mmr = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
    # faiss_retriever_sim = buff_faiss.as_retriever(search_kwargs={"k": k})
    # knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
    # knn_retriever.k = k
    bm_retriever = OktBM25Retriever.from_documents(docs)
    bm_retriever.k = k

    ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever_mmr,bm_retriever], weight=[0.5,0.5])

    
    for i, row in (dataset.iterrows()):
        if source_path != row['Source_path']:   
            source_path = row['Source_path']
            docs = faiss.similarity_search(
                query="",  # 유사도 기반이 아닌 메타데이터 필터링만 사용하므로 query는 빈 값으로
                filter={"Source_path": source_path},
                k = 99,
                fetch_k = 20000
            )
            buff_faiss = FAISS.from_documents(docs, embedding=get_embedding())
            faiss_retriever_mmr = buff_faiss.as_retriever(search_type="mmr",search_kwargs={"k": k})
            # faiss_retriever_sim = buff_faiss.as_retriever(search_kwargs={"k": k})
            # knn_retriever = KNNRetriever.from_documents(docs, embeddings=get_embedding())
            # knn_retriever.k = k
            bm_retriever = OktBM25Retriever.from_documents(docs)
            bm_retriever.k = k

            ensemble_retriever = EnsembleRetriever(retrievers=[faiss_retriever_mmr,bm_retriever], weight=[0.5,0.5])

            
        full_template = "<|begin_of_text|>"
        full_template += """<|start_header_id|>system<|end_header_id|>
당신은 유용한 금융 정보 QnA 챗봇입니다.
질문을 명확히 파악하세요. 모든 질문에 답변하세요.
정확한 답변을 반드시 요구합니다.
답변을 지어내지 마세요.
질문과 직접적인 관련이 없는 내용은 제외하세요.
간결하게 작성하세요.<|eot_id|>
"""
        question = row['Question']          
        # full_template += """ """
        contexts = ensemble_retriever.invoke(normalize_string(question))
        contexts = format_docs(contexts)
        
        full_template += "<|start_header_id|>user<|end_header_id|>"
        full_template += "Question: {input}\n\n<|eot_id|>"
        full_template += "Contexts: {contexts}"
        full_template += """<|eot_id|>"""
        full_template += """<|start_header_id|>assistant<|end_header_id>"""
        
        prompt = PromptTemplate(template=full_template, input_variables=["contexts","input"])
        qa_chain = (
        {
            "contexts": RunnablePassthrough(),
            "input": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
        )

        answer = qa_chain.invoke({'input':question,'contexts':contexts})
        answer = extract_answer(answer)
        lines = answer.split('\n')
        if  len(lines) > 1 or '|' in answer or ':' in answer or '다음과' in answer:
            previous = answer
            try:
                before = calculate_f1_score(row['Answer'],answer)[2]
            except:
                before = None
            answer = rerun(question=question,
                           context=contexts,
                           answer=answer,
                           llm=llm,
                           num_repeat=1)
        answer = extract_answer(answer)
        results.append({
            "Question": question,
            "Answer": answer,
            "Source": row['Source']
        })
        if verbose:
            print(f"{i}/{len(dataset)}")
            print("Question: ", question, end=" | ")
            print("Context Number |",len(contexts))
            try:
                print(calculate_f1_score(row['Answer'],answer)[2],end=" | ")
            except:
                pass
            print("Answer: ", results[-1]['Answer'])
            try:
                print("Before: ",before," | ",previous)  
                
                previous = None
                before = None
            except:
                pass
            
            try:
                print("REAL Answer: ",row['Answer'])
            except:
                pass
            
            print()
        torch.cuda.empty_cache()
    return results

### 케이폴드

In [15]:
# from sklearn.model_selection import KFold
# import copy

# # weight = [0.3,0.3,0.4]
# # train_faiss_db,knn_retriever ,train_bm_retrievier = make_db(train_df)

# # train_k = 3
# # train_bm_retrievier.k = train_k
# # knn_retriever.k = train_k
# # train_faiss_retriever = train_faiss_db.as_retriever(search_type="mmr",search_kwargs={'k':train_k} )
# # train_ensemble_retriever = EnsembleRetriever(
# #     retrievers=[train_bm_retrievier, knn_retriever,train_faiss_retriever], weights=weight , search_kwargs={'k':train_k}
# # )


# fewshot_k = 3

# k_folds = 4
# fold_results = []
# kf = KFold(n_splits=k_folds, shuffle=True, random_state=52)
# for fold, (train_index, val_index) in enumerate(kf.split(train_df)):
#     fold_result = []
#     # train_set = train_df.iloc[train_index]
#     val_set = train_df.iloc[val_index]

#     faiss = make_db(val_set)

#     pred = run(faiss, val_set, llm, verbose=True)
#     result = pd.DataFrame()
#     result['pred'] = [result['Answer'] for result in pred]
#     val_set.index = range(len(val_set))
#     result['gt'] = val_set['Answer']

#     result = calculate_average_f1_score(result['gt'], result['pred'])
#     print(result)
#     fold_results.append(result)
#     break

### 실전

In [16]:
def save(results):
    # 제출용 샘플 파일 로드
    submit_df = pd.read_csv("./sample_submission.csv")
    # 생성된 답변을 제출 DataFrame에 추가
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]
    time = pd.Timestamp.now()
    # 결과를 CSV 파일로 저장
    submit_df.to_csv(f"./submission_bw/sub_{time.month}_{time.day}_{time.hour}{time.minute}.csv", encoding='UTF-8-sig', index=False)
    print("저장 완료")

faiss= make_db(test_df)

results = run(faiss, test_df, llm, verbose=True)
save(results)

100%|██████████| 9/9 [00:20<00:00,  2.23s/it]


Total number of documents: 523


C:\Users\pook0\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\pook0\AppData\Roaming\Python\Python312\site-packages\transformers\models\llama\modeling_llama.py:653: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


0/98
Question:  2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요? | Context Number | 596
Answer:  2022년 혁신창업사업화자금(융자)의 예산은 2,300,000만원입니다.

1/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요? | Context Number | 864
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의 창업을 활성화하고 고용창출을 도모하고, 중소기업이 보유한 우수 기술의 상업화를 방지하고 개발기술의 제품화 및 사업화를 촉진하여 기술기반 중소기업을 육성하는 것입니다.
Before:  None  |  중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 다음과 같습니다.

1.  기술력과 사업성이 우수하고 미래 성장가능성이 높은 중소벤처기업의 창업을 활성화하고 고용창출을 도모합니다.
2.  중소기업이 보유한 우수 기술의 상업화를 방지하고 개발기술의 제품화 및 사업화를 촉진하여 기술기반 중소기업을 육성합니다.

2/98
Question:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요? | Context Number | 1335
Answer:  중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 중소기업진흥에 관한 법률 제66조, 제67조, 제74조, 중소기업창업지원법 제35조에 근거하고 있습니다.
Before:  None  |  None

3/98
Question:  2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요? | Context Number | 714
Answer:  2010년에 신규 지원된 혁신창업사업화자금은 재창업자금(실패 경영인에 대한 재기지원)이란 것입니다.
Before:  None  |  None

4/98
Question:  혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요? | Context Number | 714


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


8/98
Question:  혁신창업사업화자금(융자) 사업 집행절차는 어떻게 되나요? | Context Number | 975
Answer:  혁신창업사업화자금(융자) 사업 집행절차는 사전상담 및 신청, 사업계획수립/공고, 융자 실행 지원결정통보 평가 및 승인, 융자 실행 순입니다.
Before:  None  |  혁신창업사업화자금(융자) 사업 집행절차는 다음과 같습니다.

1. 사전상담 및 신청
2. 사업계획수립/공고
3. 융자 실행 지원결정통보 평가 및 승인
4. 융자 실행



C:\Users\pook0\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


9/98
Question:  부모급여 지원 사업의 목적은 무엇인가요? | Context Number | 1091
Answer:  부모급여 지원 사업의 목적은 출산 및 양육으로 손실되는 소득을 보전하고, 주 양육자의 직접 돌봄이 중요한 아동 발달의 특성에 따라 영아기 돌봄을 두텁게 지원하기 위해 부모급여 지급입니다.
Before:  None  |  None

10/98
Question:  부모급여(영아수당)의 2024년 확정된 예산은 몇백만원인가요? | Context Number | 993
Answer:  2024 년 부모급여(영아수당)의 확정된 예산은 2,888,694 백만원입니다.
Before:  None  |  None

11/98
Question:  부모급여 지원 사업은 어떤 법령상 근거를 갖고 추진되고 있나요? | Context Number | 1349
Answer:  부모급여 지원 사업은 아동수당법 제4조 제5항에 근거하고 있습니다.
Before:  None  |  None

12/98
Question:  영아수당 도입에 대한 추진경위는 어떻게 되나요? | Context Number | 1110
Answer:  영아수당 도입의 추진경위는 예비타당성조사 통과, 근거법 마련, 영아수당 지원사업 시행, 대통령 공약사항 및 국정과제 포함, 부모급여 지급으로 가정양육 지원 및 부모의 경제적 부담 완화 등으로 진행되었습니다.
Before:  None  |  영아수당 도입에 대한 추진경위는 다음과 같습니다.

1.  예비타당성조사 통과(2021년 8월)
2.  근거법 마련(2021년 12월)
3.  영아수당 지원사업 시행(2022년 1월부터)
4.  '부모급여 도입' 대통령 공약사항 및 국정과제에 포함(2022년 5월)
5.  0~11개월 아동에 대한 부모급여 지급으로 가정양육 지원 및 부모의 경제적 부담을 획기적으로 완화(2023년 70만원, 2024년 100만원으로 단계적 확대)

이러한 추진경위로 부모급여(영아수당) 지원 사업이 추진되고 있습니다.

13/9

C:\Users\pook0\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


15/98
Question:  노인장기요양보험 사업 운영에 대한 목적은 무엇인가요? | Context Number | 1364
Answer:  노인장기요양보험 사업 운영의 목적은 노인이나 고령자들이 일상생활을 혼자서 수행하기 어려운 경우 신체 또는 가사 활동을 제공하는 노인장기요양보험에 대한 국고 지원을 통해 효율적인 정책 추진을 통해 노후의 건강 증진 및 생활 안정 도모와 가족의 부담 완화 및 국민 삶의 질 향상을 목적으로 합니다.
Before:  None  |  None

